In [1]:
import numpy as np

# problem 1

In the steepest descent method,
$$
\begin{aligned}
<v^{(k+1)},v^{(k)}>&=<r^{(k+1)},r^{(k)}>=<r^{(k)}-t_k A r^{(k)},r^{(k)}>=<r^{(k)},r^{(k)}>-t_k<r^{(k)},Ar^{(k)}>\\
&=<r^{(k)},r^{(k)}>-\frac{<r^{(k)},r^{(k)}>}{<r^{(k)},A r^{(k)}>}<r^{(k)},A r^{(k)}>=<r^{(k)},r^{(k)}>-<r^{(k)},r^{(k)}>\\
&=0
\end{aligned}
$$
Thus the consecutive search directions are orthogonal.

# problem 2

In the conjugate gradient method,
$$
v^{(k)}=r^{(k)}+s_k v^{(k-1)}
$$
If $v^{(k)}=0$, then
$$
\begin{aligned}
& r^{(k)}+s_k v^{(k-1)}=0\\
\Rightarrow & <r^{(k)},r^{(k)}>+s_k<r^{(k)}, v^{(k-1)}>=0\\
\Rightarrow & <r^{(k)},r^{(k)}>=0\\
\Rightarrow & r^{(k)}=0 \\
\Rightarrow & b-A x^{(k)}=0 \\
\Rightarrow & A x^{(k)}=b
\end{aligned}
$$


# problem 3

Implement the conjugate gradient method

In [2]:
def cg(A, b, TOL):
    """
    This function solve the equation Ax = b where A is a symmetric
    positive definite matrix by the conjugate gradient method.
    
    Inputs:
        A: symmetric positive definite matrix of size n-by-n
        b: ordinate of size n
        TOL: tolerance

    Output:
        x: solution of the equation
        k: number of iterations
        error: error in 2-norm
    """
    x = np.zeros_like(b, dtype='double')
    r = b - A.dot(x)
    v = r
    k = 0
    
    d = r.dot(r)
    
    while np.sqrt(d) > TOL:
        t = d/v.dot(A.dot(v))
        x = x + t*v
        r = r - t*A.dot(v)
        dp = r.dot(r)
        v = r + dp/d*v
        d = dp
        k = k + 1
    
    error = np.linalg.norm(b-A.dot(x), 2)
    
    return x, k, error

Solve the system for $N = 50$, $N = 100$ and $N = 200$, check convergence and comment on the required number of iterations.

In [3]:
TOL = 1E-9  # tolerance

for N in (50, 100, 200):
    h = 1/N  # grid size
    x = h*np.arange(N+1)
        
    # the linear system
    A = np.diag(-np.ones(N-2)/h**2, -1)+ np.diag(np.ones(N-1)*(2/h**2+np.pi**2)) \
        + np.diag(-np.ones(N-2)/h**2, 1)
    b = 2*np.pi**2*np.sin(np.pi*x[1:N])
    
    # solve the system
    v = np.zeros(N+1)
    v[1:N], k, error = cg(A, b, TOL)
    
    print('At N = {}, #iterations = {}, error = {}'.format(N, k, error))

At N = 50, #iterations = 1, error = 4.1640892186447175e-12
At N = 100, #iterations = 1, error = 2.983227862114191e-11
At N = 200, #iterations = 1, error = 1.1783425929129354e-10


For symmetric positive definite matrix $A$, the results quickly converge to the solution with $TOL$ in the conjugate gradient method. Despite the round-off error, the required number of iterations is at most $N$.

 Compare with the performance of Jacobi's method.

In [4]:
# implement the jacobi method.
def jacobi(A, b, TOL):
    """
    This function solve the equation Ax = b by the jacobi's method.
    
    Inputs:
        A: symmetric positive definite matrix of size n-by-n
        b: ordinate of size n
        TOL: tolerance

    Output:
        x: solution of the equation
        k: number of iterations
        error: error in 2-norm
    """   
    D = np.diag(A.diagonal())
    Dinv = np.linalg.inv(D)
    
    # iterative matrix
    T = -Dinv.dot(A-D)
    c = Dinv.dot(b)
    
    x = np.zeros_like(b, dtype='double')
    k = 0
    error = np.linalg.norm(b-A.dot(x), 2) 
    
    while error > TOL:
        x = T.dot(x)+c
        k = k + 1
        error = np.linalg.norm(b-A.dot(x), 2) 
        
    return x, k, error

TOL = 1E-9  # tolerance

for N in (50, 100, 200):
    h = 1/N  # grid size
    x = h*np.arange(N+1)
        
    # the linear system
    A = np.diag(-np.ones(N-2)/h**2, -1)+ np.diag(np.ones(N-1)*(2/h**2+np.pi**2)) \
        + np.diag(-np.ones(N-2)/h**2, 1)
    b = 2*np.pi**2*np.sin(np.pi*x[1:N])
    
    # solve the system
    v = np.zeros(N+1)
    v[1:N], k, error = jacobi(A, b, TOL)
    
    print('At N = {}, #iterations = {}, error = {}'.format(N, k, error))

At N = 50, #iterations = 6414, error = 9.975836782478501e-10
At N = 100, #iterations = 26023, error = 9.97369162961708e-10
At N = 200, #iterations = 105228, error = 9.972676494670398e-10


We see the required number of iterations of Jacobi's is large and increases rapidly as $N$ increases. But the Jacobi method can solve systems that are not positive definite. 

